<h1> HealthCare Analysis Using Medicare Data </h1>
<h2> CS 418 - The Data Squad</h2>

This notebook will take care of any data cleaning and answer the following questions:
<ul>
    <li> What data do we have and what data do we need? </li>
    <li> How will we collect more data? </li>
    <li> How do we organize the data for analysis? </li>
</ul>
First, Import modules.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import helper_functions as hf
%matplotlib inline
import matplotlib.pyplot as plt
from datetime import datetime

<h3> What data do we have? What do we need? </h3>
<br> We will use the following files: <br>
<ul>
    <li>
        <a href="https://go.cms.gov/2WyRXft" > 
        Medicare_Provider_Charge_Inpatient_DRGALL_FY2016.csv 
        </a>
    </li>
    <li> 
        <a href="https://go.cms.gov/2HSD2Ze">
            Medicare_OPPS_CY2016_Provider_APC.xlsx 
        </a>
    </li>
    <li> Add more files here... </li>
</ul>

In [2]:
in_charges_path = "..\data_files\Medicare_Provider_Charge_Inpatient_DRGALL_FY2016.csv"
in_charges_summary_path = "..\data_files\Medicare_Charge_Inpatient_DRGALL_DRG_Summary_Reports_FY2016.xlsx"
out_charges_path = "..\data_files\Medicare_OPPS_CY2016_Provider_APC.xlsx"
out_charges_summary_path = "..\data_files\Medicare_OPPS_CY2016_Summary_APC_HCPCS.xlsx"

Here is some information on the first file.

In [3]:
# Read the csv file
in_charges = pd.read_csv(in_charges_path, skipinitialspace=True, low_memory=False)

# Print colums and their meanings
# This info comes from the 2nd sheet of the summary file
# Starting at row 4, col 2
pd.set_option('display.max_colwidth', -1)
in_summary_xls = pd.ExcelFile(in_charges_summary_path)
in_summary = pd.read_excel(in_summary_xls, "Data Dictionary", 
                           skiprows = 2, index_col = 0)
in_summary

,Description
Short Name,
DRG Definition,Classification system that groups similar clinical conditions (diagnoses) and the procedures furnished by the hospital during the stay.
Provider State,State in which the providers are physically located.
Total Discharges,The number of discharges billed by all providers for inpatient hospital services.
Average Covered Charges,The average charge of all provider's services covered by Medicare for discharges in the DRG. These will vary from hospital to hospital because of differences in hospital charge structures.
Average Total Payments,"The average total payments to all providers for the DRG including the MS-DRG amount, teaching, disproportionate share, capital, and outlier payments for all cases. Also included in average total payments are co-payment and deductible amounts that the patient is responsible for and any additional payments by third parties for coordination of benefits."
Average Medicare Payments,"The average amount that Medicare pays to the provider for Medicare's share of the MS-DRG. Medicare payment amounts include the MS-DRG amount, teaching, disproportionate share, capital, and outlier payments for all cases. Medicare payments DO NOT include beneficiary co-payments and deductible amounts nor any additional payments from third parties for coordination of benefits."


Each row in the in_charges dataframe represents:
<blockquote> The Inpatient Utilization and Payment Public Use File (Inpatient PUF) provides information on inpatient discharges for Medicare fee-for-service beneficiaries. </blockquote>

What is Medicare fee for service?
<blockquote> Medicare Fee-for-Service (FFS) is a program that provides hospital insurance (Part A) and supplementary medical insurance (Part B) to eligible citizens. In general, Part A (hospital insurance) covers inpatient hospital care, skilled nursing facility, hospice, lab tests, surgery, and home health care. Part B (medical insurance) covers services including physician and other health care providers' services, outpatient care, durable medical equipment, and some preventive services. </blockquote>

In [4]:
print(in_charges.dtypes)
hf.df_interact(in_charges)

DRG Definition                                object
Provider Id                                   int64 
Provider Name                                 object
Provider Street Address                       object
Provider City                                 object
Provider State                                object
Provider Zip Code                             int64 
Hospital Referral Region (HRR) Description    object
Total Discharges                              object
Average Covered Charges                       object
Average Total Payments                        object
Average Medicare Payments                     object
dtype: object


interactive(children=(IntSlider(value=0, description='row', max=197283, step=5), IntSlider(value=0, descriptio…

(197283 rows, 12 columns) total


We have to be careful using these columns because they contain null values.

In [5]:
null_cols = in_charges.isnull().sum()
in_charges.columns[null_cols > 0]

Index([], dtype='object')

Now, do the same for the out patient charges

In [6]:
# Read the csv file and call the interact function to 
# get a glimpse of the data, skip 5 header rows
out_charges = pd.read_excel(out_charges_path, skipinitialspace=True, skiprows = 5)

# Print colums and their meanings
# This info comes from the 2nd sheet of the summary file
# Starting at row 4, col 2
out_summary_xls = pd.ExcelFile(out_charges_summary_path)
out_summary = pd.read_excel(out_summary_xls, "Data Dictionary", 
                           skiprows = 3, usecols='B:C', index_col = 0,
                           skipfooter = 2)
out_summary

,Description
Column Name,
APC,"The comprehensive APC code. In 2016, CMS renumbered the APC codes. Therefore, CY 2015 APC codes have different values from the CY 2016 APC codes. Appendix Table 1 in the Outpatient Hospital PUF methodology report contains a crosswalk between the CY 2015 and CY 2016 APC codes."
APC Description,The description of the APC Code.
Primary HCPCS,The HCPCS code for the primary HCPCS that is the basis for setting payment for the APC group.
Primary HCPCS Description,The HCPCS code description for the primary HCPCS that is the basis for setting payment for the APC group.
Comprehensive APC Services,The number of primary HCPCS services billed by the provider for outpatient hospital services. For the APC/HCPCS level summary it is a count of all the primary services for a given APC-primary HCPCS combination.
Average Estimated Total Submitted Charges,The provider's average estimated submitted charge for services covered by Medicare for the APC-primary HCPCS combination. These will vary from hospital to hospital because of differences in hospital charge structures.
Average Medicare Allowed Amount,"The average of total regular payments the provider receives for the APC-primary HCPCS. It includes both Medicare regular provider payments as well as beneficiaries’ co-payment and deductible payments. It excludes special outlier payments which are reported in a separate column. The APC allowed amounts do not vary by primary HCPCS. However, slight variations will occur based on geographic adjustments based on where the services are delivered."
Average Medicare Payment Amount,"The average of total Medicare regular payments the provider receives directly from Medicare. It excludes special outlier payments which are reported in a separate column. The APC payments do not vary by primary HCPCS. However, slight variations will occur based on geographic adjustments based on where the services are delivered."
Outlier Comprehensive APC Services,The number of comprehensive APC services with outlier payments. This variable is blank in cases where the number of outlier services is fewer than 11.


Here is the data:
<blockquote> The Outpatient Hospital Utilization and Payment Public Use File (Outpatient Hospital PUF) presents information on comprehensive APC (C-APC) services provided to Medicare fee-for-service beneficiaries. </blockquote>

<b> APCs </b> or Ambulatory Payment Classifications are the United States government's method of paying for facility outpatient services for the Medicare (United States) program. APC payments are made to hospitals when the Medicare outpatient is discharged from the Emergency Department or clinic or is transferred to another hospital (or other facility) which is not affiliated with the initial hospital where the patient received outpatient services.

In [8]:
print(out_charges.dtypes)
hf.df_interact(out_charges)

Provider ID                                      int64  
Provider Name                                    object 
Provider Street Address                          object 
Provider City                                    object 
Provider\nState                                  object 
Provider\nZip Code                               int64  
Provider\nHospital Referral Region\n(HRR)        object 
APC                                              int64  
APC\nDescription                                 object 
Comprehensive APC\nServices                      int64  
Average\nEstimated\nTotal\nSubmitted\nCharges    float64
Average\nMedicare\nAllowed\nAmount               float64
Average\nMedicare\nPayment\nAmount               float64
Outlier\nComprehensive\nAPC\nServices            float64
Average\nMedicare\nOutlier\nAmount               float64
dtype: object


interactive(children=(IntSlider(value=0, description='row', max=27652, step=5), IntSlider(value=0, description…

(27652 rows, 15 columns) total


In [54]:
# We have to be careful using these columns because they
# contain null values
null_cols = out_charges.isnull().sum()
out_charges.columns[null_cols > 0]

Index(['Outlier\nComprehensive\nAPC\nServices', 'Average\nMedicare\nOutlier\nAmount'], dtype='object')

<b> Granularity <b>

Inpatient Charges
<br> <b> Summary Reports: </b> We present discharges, average total covered charges, average total payments and average Medicare payments by DRG, both nationally and by state.  Any aggregated records  which are derived from 10 or fewer discharges are excluded to protect the privacy of Medicare beneficiaries. </br>

In [40]:
"""
Hospital Referral Region (HRR) Description seems to have
unnecessary state abbreviation.
We need to double check state abbreviation matches, so we
can get rid of everything before hyphen.
"""

HRR_states = in_charges['Hospital Referral Region (HRR) Description'].astype(str).str[0:2]
rows = in_charges[in_charges['Provider State'] != HRR_states]
rows[['Provider State', 'Hospital Referral Region (HRR) Description']].head()

,Provider State,Hospital Referral Region (HRR) Description
223,ID,WA - Spokane
255,KS,MO - Kansas City
325,MS,TN - Memphis
364,NJ,PA - Philadelphia
657,GA,FL - Jacksonville


Outpatient Charges
<br> <b> Summary Metrics: </b> CMS presents the number of APC services, hospitals' average total estimated submitted charges, the average Medicare allowed charges (which includes Medicare regular provider payments and beneficiary cost-sharing payments), the average Medicare regular provider payments, the number of APC services with outlier payments, and the average Medicare outlier provider payments among those services. Any aggregated records which are derived from fewer than 11 APC primary HCPCS services are excluded to protect the privacy of Medicare beneficiaries.</br>

In [38]:
"""
Similarly Provide Hospital Referral Region (HRR)
"""

HRR_states = out_charges['Provider\nHospital Referral Region\n(HRR)'].astype(str).str[0:2]
rows = out_charges[out_charges['Provider\nState'] != HRR_states]
rows[['Provider\nState','Provider\nHospital Referral Region\n(HRR)']].head()


,Provider State,Provider Hospital Referral Region (HRR)
91,AL,GA - Rome
92,AL,GA - Rome
93,AL,GA - Rome
178,AL,FL - Pensacola
179,AL,FL - Pensacola


<b> Scope <b>

Inpatient Charges
<br> Study Population: Medicare Inpatient Prospective Payment System (IPPS) providers within the 50 United States and District of Columbia with a known Hospital Referral Region (HRR) who are billing Medicare fee-for-service. </br>

<br> <b> Limitations of Maryland Data: </b> The state of Maryland has a unique waiver that exempts it from Medicare’s prospective payment systems for inpatient care.  Maryland instead uses an all-payer rate setting commission to determine its payment rates.  Medicare claims for hospitals in other states break out additional payments for indirect medical education (IME) costs and disproportionate share hospital (DSH) adjustments. </br>

Outpatient Charges
<br> Study Population: Medicare Outpatient Prospective Payment System (OPPS) providers within 49 of the 50 United States and District of Columbia  (excluding Maryland) with a known Hospital Referral Region (HRR) who are billing Medicare fee-for-service beneficiaries for the comprehensive APCs (C-APC).   </br>

<b> Temporality <b>

Inpatient Charges
<br> Years: Fiscal Year 2016 </br>

Outpatient Charges
<br> Years: Calendar Year 2016 </br>

<b> Faithfulness <b>

// TODO: We describe a dataset as "faithful" if we believe it accurately captures reality. Typically, untrustworthy datasets contain:

<h3> How will we collect more data? </h3>


<h3> How do we organize the data for analysis? <3>

For the in patient charges, we have to convert the following columns to ints:
<ul>
    <li> Total Discharges                              object </li>
    <li> Average Covered Charges                       object </li>
    <li> Average Total Payments                        object </li>
    <li> Average Medicare Payments                     object </li>
   
</ul>
As well as removing everything before the hyphen in the following columns:
<ul>
    <li> DRG Definition                                object </li>
</ul>

For the out patient charges, we have to change the names for the following columns:
<ul>
    <li> Provider\nState                                  object </li>
    <li> Provider\nZip Code                               int64 </li>
    <li> Provider\nHospital Referral Region\n(HRR)        object </li>
    <li> APC\nDescription                                 object </li>
    <li> Comprehensive APC\nServices                      int64  </li>
    <li> Average\nEstimated\nTotal\nSubmitted\nCharges    float64 </li>
    <li> Average\nMedicare\nAllowed\nAmount               float64 </li>
    <li> Average\nMedicare\nPayment\nAmount               float64 </li>
    <li> Outlier\nComprehensive\nAPC\nServices            float64 </li>
    <li> Average\nMedicare\nOutlier\nAmount               float64 </li>
   
</ul>